In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import warnings
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
#verinin tanımlanması
data = pd.read_excel('/content/zemberek.xlsx')
data.head()

,comments,label,Köklü yorumlar
0,gerçek siyaset böyle yapılmalı sinirlenmeden ü...,mutluluk,gerçek siyaset böyle yap sinir üslup saygı dur...
1,ömrüm bin yıl olsa bıkmadan duraklamadan izler...,mutluluk,ömür bin yıl ol bık durakla izle barış be yol ...
2,çıkan konuklar arasında keyifli sıkılmadan izl...,mutluluk,çık konuk ara keyif sıkıl izle bölüm ol
3,bütün mam serisinde begendigim hatta bilgisaya...,mutluluk,bütün mam seri UNK hatta bilgisayar UNK tek bö...
4,burdan çıkamıyorum defalarca izliyorum hala do...,mutluluk,bura çık defalarca izle hala doy içten samimi ...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18211 entries, 0 to 18210
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   comments        18197 non-null  object
 1   label           18211 non-null  object
 2   Köklü yorumlar  18195 non-null  object
dtypes: object(3)
memory usage: 426.9+ KB


In [ ]:
data.shape

(18211, 3)

In [ ]:
class_ = Counter(data['label']).keys()
class_

dict_keys(['mutluluk', 'üzüntü', 'korku', 'öfke', 'şaşkınlık', 'küçümseme'])

In [ ]:
import math
dataDoc = data['comments'].values.tolist()
dataClass = data['label'].values.tolist()
#test ve train olarak verinin ayrılması
x_train, x_test, y_train, y_test = train_test_split(dataDoc, dataClass, test_size = 0.3, random_state = 42)
#tfidf işlemi
## max_df 0,9 tfidf skorundan fazla olanları kelimeleri alma
## min_df 5 frekansından düşük olan kelimeleri alma
dataDoc = ["" if isinstance(doc, float) and math.isnan(doc) else doc for doc in dataDoc]
x_train = ["" if isinstance(doc, float) and math.isnan(doc) else doc for doc in x_train]
x_test = ["" if isinstance(doc, float) and math.isnan(doc) else doc for doc in x_test]
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.8, min_df=1)
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf = tfidf_vectorizer.transform(x_test)
print("Number of documents in x_train_tfidf:", x_train_tfidf.shape)
print("Number of documents in x_test_tfidf:", x_test_tfidf.shape)

Number of documents in x_train_tfidf: (12747, 145294)
Number of documents in x_test_tfidf: (5464, 145294)


In [ ]:
words_counts = Counter([word for line in x_train for word in line.split(' ')])
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:100]
most_common_words[:100]

[('bir', 4886),
 ('kadar', 1318),
 ('beni', 859),
 ('ben', 744),
 ('zaman', 515),
 ('var', 514),
 ('gerçekten', 508),
 ('mutlu', 486),
 ('iyi', 436),
 ('sonra', 431),
 ('güzel', 431),
 ('bile', 430),
 ('bana', 430),
 ('şaşırdım', 420),
 ('', 403),
 ('böyle', 375),
 ('değil', 337),
 ('mi', 330),
 ('yok', 330),
 ('hüzün', 330),
 ('artık', 315),
 ('olsun', 314),
 ('oldum', 309),
 ('korkarım', 308),
 ('benim', 304),
 ('olduğunu', 284),
 ('gün', 280),
 ('büyük', 280),
 ('olan', 275),
 ('oldu', 261),
 ('yine', 259),
 ('sen', 259),
 ('olarak', 258),
 ('korkuyorum', 250),
 ('korktum', 244),
 ('an', 244),
 ('insan', 242),
 ('senin', 240),
 ('ilk', 238),
 ('tek', 236),
 ('bi', 220),
 ('sadece', 217),
 ('seni', 215),
 ('bunu', 213),
 ('allah', 202),
 ('kendi', 200),
 ('aynı', 197),
 ('yeni', 195),
 ('adam', 193),
 ('görünce', 190),
 ('korkmuştum', 183),
 ('ediyorum', 182),
 ('kötü', 180),
 ('gece', 180),
 ('sürpriz', 177),
 ('önce', 175),
 ('tam', 175),
 ('beklemediğim', 174),
 ('başka', 166),
 (

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier  # Importing MLPClassifier
import matplotlib.pyplot as plt
import sklearn


#logistic regresyon
model = OneVsRestClassifier(LogisticRegression(penalty = 'l2', C=1.0))
model.fit(x_train_tfidf, y_train)

print ("Logistic Regression Accuracy={}".format(accuracy_score(y_test, model.predict(x_test_tfidf))))
logisticpred = accuracy_score(y_test, model.predict(x_test_tfidf))
# F score, precision ve recall değerlerini hesaplayalım

f_score = f1_score(y_test, model.predict(x_test_tfidf), average='macro')
precision = precision_score(y_test, model.predict(x_test_tfidf),average='macro')
recall = recall_score(y_test, model.predict(x_test_tfidf), average='macro')
print("F score:", f_score)
print("Precision:", precision)
print("Recall:", recall)


print("   ")

# Naive Bayes
naive_bayes = MultinomialNB()
naive_bayes.fit(x_train_tfidf, y_train)
nb_pred = naive_bayes.predict(x_test_tfidf)

print('Naive Bayes Accuracy: {}'.format(accuracy_score(y_test, nb_pred)))
print("F Score:", f1_score(y_test, nb_pred, average='macro'))
print("Precision:", precision_score(y_test, nb_pred, average='macro'))
print("Recall:", recall_score(y_test, nb_pred, average='macro'))



print(" ")
# Support Vector Machine (SVM)
svm = OneVsRestClassifier(SVC(kernel='linear'))
svm.fit(x_train_tfidf, y_train)
svm_pred = svm.predict(x_test_tfidf)

print('SVM Accuracy: {}'.format(accuracy_score(y_test, svm_pred)))
print("F Score:", f1_score(y_test, svm_pred, average='macro'))
print("Precision:", precision_score(y_test, svm_pred, average='macro'))
print("Recall:", recall_score(y_test, svm_pred, average='macro'))

print(" ")


# Decision Tree
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train_tfidf, y_train)
dt_pred = decision_tree.predict(x_test_tfidf)



print('Decision Tree Accuracy: {}'.format(accuracy_score(y_test, dt_pred)))
print("F Score:", f1_score(y_test, dt_pred, average='macro'))
print("Precision:", precision_score(y_test, dt_pred, average='macro'))
print("Recall:", recall_score(y_test, dt_pred, average='macro'))

print(" ")

# Random Forest
random_forest = RandomForestClassifier()
random_forest.fit(x_train_tfidf, y_train)
rf_pred = random_forest.predict(x_test_tfidf)



print('Random Forest Accuracy: {}'.format(accuracy_score(y_test, rf_pred)))
print("F Score:", f1_score(y_test, rf_pred, average='macro'))
print("Precision:", precision_score(y_test, rf_pred, average='macro'))
print("Recall:", recall_score(y_test, rf_pred, average='macro'))



# Multi layer perceptron (MLP)
mlp = MLPClassifier()
mlp.fit(x_train_tfidf, y_train)
mlp_pred = mlp.predict(x_test_tfidf)



print('MLP Accuracy: {}'.format(accuracy_score(y_test, mlp_pred)))
print("F Score:", f1_score(y_test, mlp_pred, average='macro'))
print("Precision:", precision_score(y_test, mlp_pred, average='macro'))
print("Recall:", recall_score(y_test, mlp_pred, average='macro'))

Logistic Regression Accuracy=0.7999633967789166
F score: 0.8005763113730855
Precision: 0.8058215319777684
Recall: 0.7982330216214164
   
Naive Bayes Accuracy: 0.8113103953147877
F Score: 0.8089025017082223
Precision: 0.8128829946739614
Recall: 0.809204722961641
 
SVM Accuracy: 0.8308931185944363
F Score: 0.8291784234337817
Precision: 0.8304065241940065
Recall: 0.8290369650730641
 
Decision Tree Accuracy: 0.7015007320644217
F Score: 0.6994741406761076
Precision: 0.7057084100216559
Recall: 0.6999030817797155
 
Random Forest Accuracy: 0.763909224011713
F Score: 0.7632533330423366
Precision: 0.7673853724389278
Recall: 0.762324077798514
MLP Accuracy: 0.8308931185944363
F Score: 0.8289658699272279
Precision: 0.8304798584686583
Recall: 0.8291135806484906


In [ ]:
#Training ML classification models

# Naive Bayes
from sklearn.naive_bayes import GaussianNB, MultinomialNB


# MultinomialNB is used in multi-class classification
#clf = Gaussian#NB()
clf = MultinomialNB()


print(clf)

NB=clf.fit(x_train_tfidf,y_train)
prediction=NB.predict(x_test_tfidf)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test, prediction, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

# Importing cross validation function from sklearn
#from sklearn.model_selection import cross_val_score

# Running 10-Fold Cross validation on a given algorithm
# Passing full data X and y because the K-fold will split the data and automatically choose train/test
#Accuracy_Values=cross_val_score(NB, X , y, cv=10, scoring='f1_weighted')
#print('\nAccuracy values for 5-fold Cross Validation:\n',Accuracy_Values)
#print('\nFinal Average Accuracy of the model:', round(Accuracy_Values.mean(),2))

MultinomialNB()
              precision    recall  f1-score   support

       korku       0.94      0.87      0.91       921
   küçümseme       0.77      0.64      0.70       877
    mutluluk       0.78      0.82      0.80       908
        öfke       0.79      0.74      0.76       895
      üzüntü       0.69      0.85      0.76       920
   şaşkınlık       0.90      0.91      0.91       943

    accuracy                           0.81      5464
   macro avg       0.81      0.80      0.80      5464
weighted avg       0.81      0.81      0.81      5464

[[805  10  12  25  43  26]
 [ 19 558  94  70 112  24]
 [  9  39 747  25  71  17]
 [  9  57  40 659 112  18]
 [  5  39  55  32 778  11]
 [ 11  22  14  18  17 861]]
Accuracy of the model on Testing Sample Data: 0.81


In [ ]:
from sklearn.svm import SVC

# Choose hyperparameters (you can tune these)
svm_clf = SVC(kernel='linear', C=1.0)

# Printing all the parameters of SVM
print(svm_clf)

# Creating the model on Training Data
svm_model = svm_clf.fit(x_train_tfidf, y_train)
prediction_svm = svm_model.predict(x_test_tfidf)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test, prediction_svm))
print(metrics.confusion_matrix(y_test, prediction_svm))

# Printing the Overall Accuracy of the model
f1_score_svm = metrics.f1_score(y_test, prediction_svm, average='weighted')
print('Accuracy of the SVM model on Testing Sample Data:', round(f1_score_svm, 2))

# Importing cross-validation function from sklearn
from sklearn.model_selection import cross_val_score

# Running 10-Fold Cross-validation on the SVM model
# Passing full data X and y because the K-fold will split the data and automatically choose train/test
#accuracy_values_svm = cross_val_score(svm_model, X, y, cv=10, scoring='f1_weighted')
#print('\nAccuracy values for 10-fold Cross Validation:\n', accuracy_values_svm)
#print('\nFinal Average Accuracy of the SVM model:', round(accuracy_values_svm.mean(), 2))

SVC(kernel='linear')
              precision    recall  f1-score   support

       korku       0.96      0.92      0.94       921
   küçümseme       0.71      0.72      0.72       877
    mutluluk       0.83      0.83      0.83       908
        öfke       0.80      0.77      0.78       895
      üzüntü       0.75      0.82      0.78       920
   şaşkınlık       0.96      0.93      0.95       943

    accuracy                           0.83      5464
   macro avg       0.84      0.83      0.83      5464
weighted avg       0.84      0.83      0.84      5464

[[846  17   5  19  23  11]
 [ 10 634  73  72  83   5]
 [  4  68 754  25  52   5]
 [  9  79  23 690  83  11]
 [  5  72  43  40 756   4]
 [  7  19   8  18  14 877]]
Accuracy of the SVM model on Testing Sample Data: 0.84


In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Choose hyperparameters (you can tune these)
clf = RandomForestClassifier(n_estimators=600, max_depth=40, criterion='gini')

# Printing all the parameters of Random Forest
print(clf)

# Creating the model on Training Data
RF_model = clf.fit(x_train_tfidf, y_train)
prediction = RF_model.predict(x_test_tfidf)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

# Printing the Overall Accuracy of the model
F1_Score = metrics.f1_score(y_test, prediction, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score, 2))



# Importing cross-validation function from sklearn
from sklearn.model_selection import cross_val_score

# Running 10-Fold Cross-validation on a given algorithm
# Passing full data X and y because the K-fold will split the data and automatically choose train/test
#Accuracy_Values = cross_val_score(RF_model, X, y, cv=10, scoring='f1_weighted')
#print('\nAccuracy values for 10-fold Cross Validation:\n', Accuracy_Values)
#print('\nFinal Average Accuracy of the model:', round(Accuracy_Values.mean(), 2))


RandomForestClassifier(max_depth=40, n_estimators=600)
              precision    recall  f1-score   support

       korku       0.89      0.90      0.89       921
   küçümseme       0.59      0.70      0.64       877
    mutluluk       0.78      0.76      0.77       908
        öfke       0.78      0.69      0.73       895
      üzüntü       0.73      0.73      0.73       920
   şaşkınlık       0.92      0.90      0.91       943

    accuracy                           0.78      5464
   macro avg       0.78      0.78      0.78      5464
weighted avg       0.79      0.78      0.78      5464

[[825  28   4  27  22  15]
 [ 31 612  79  64  75  16]
 [ 17 105 686  31  54  15]
 [ 17 119  49 616  75  19]
 [ 26 123  53  42 669   7]
 [  8  51   5  11  16 852]]
Accuracy of the model on Testing Sample Data: 0.78


In [ ]:
# Decision Trees
from sklearn import tree
#choose from different tunable hyper parameters
clf = tree.DecisionTreeClassifier(max_depth=20,criterion='gini')

# Printing all the parameters of Decision Trees
print(clf)

# Creating the model on Training Data
DTree=clf.fit(x_train_tfidf,y_train)
prediction=DTree.predict(x_test_tfidf)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test, prediction, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))



# Importing cross validation function from sklearn
#from sklearn.model_selection import cross_val_score

# Running 10-Fold Cross validation on a given algorithm
# Passing full data X and y because the K-fold will split the data and automatically choose train/test
#Accuracy_Values=cross_val_score(DTree, X , y, cv=10, scoring='f1_weighted')
#print('\nAccuracy values for 10-fold Cross Validation:\n',Accuracy_Values)
#print('\nFinal Average Accuracy of the model:', round(Accuracy_Values.mean(),2))

DecisionTreeClassifier(max_depth=20)
              precision    recall  f1-score   support

       korku       0.99      0.56      0.72       921
   küçümseme       0.23      0.98      0.38       877
    mutluluk       0.93      0.31      0.46       908
        öfke       0.95      0.24      0.38       895
      üzüntü       0.89      0.09      0.17       920
   şaşkınlık       0.98      0.66      0.79       943

    accuracy                           0.47      5464
   macro avg       0.83      0.47      0.48      5464
weighted avg       0.83      0.47      0.48      5464

[[516 401   0   0   2   2]
 [  1 863   6   2   0   5]
 [  0 622 278   0   4   4]
 [  2 673   3 211   4   2]
 [  2 820   5   7  84   2]
 [  0 313   7   1   0 622]]
Accuracy of the model on Testing Sample Data: 0.48


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import pandas as pd

# Choosing hyperparameters for Logistic Regression
clf = LogisticRegression(C=25, penalty='l2', solver='saga')

# Training the model on training data
LOG = clf.fit(x_train_tfidf, y_train)

# Generating predictions on testing data
prediction = LOG.predict(x_test_tfidf)

# Creating feature names for the DataFrame
feature_names = [f'Feature_{i}' for i in range(x_test_tfidf.shape[1])]

# Printing sample values of predictions in testing data
TestingData = pd.DataFrame(data=x_test_tfidf.toarray(), columns=feature_names)
TestingData['Actual'] = y_test
TestingData['Predicted'] = prediction
print(TestingData.head())

# Measuring accuracy on testing data
print("Classification Report:")
print(classification_report(y_test, prediction))

print("Confusion Matrix:")
print(confusion_matrix(y_test, prediction))

# Calculating F1 Score
f1 = f1_score(y_test, prediction, average='weighted')
print('F1 Score:', round(f1, 2))

# Calculating Accuracy
accuracy = accuracy_score(y_test, prediction)
print('Accuracy:', round(accuracy, 2))


## Importing cross validation function from sklearn
#from sklearn.model_selection import cross_val_score

## Running 10-Fold Cross validation on a given algorithm
## Passing full data X and y because the K-fold will split the data and automatically choose train/test
#Accuracy_Values=cross_val_score(LOG, X , y, cv=10, scoring='f1_weighted')
#print('\nAccuracy values for 10-fold Cross Validation:\n',Accuracy_Values)
#print('\nFinal Average Accuracy of the model:', round(Accuracy_Values.mean(),2))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


   Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0   
2        0.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0   

   Feature_6  Feature_7  Feature_8  Feature_9  ...  Feature_118868  \
0        0.0        0.0        0.0        0.0  ...             0.0   
1        0.0        0.0        0.0        0.0  ...             0.0   
2        0.0        0.0        0.0        0.0  ...             0.0   
3        0.0        0.0        0.0        0.0  ...             0.0   
4        0.0        0.0        0.0        0.0  ...             0.0   

   Feature_118869  Feature_118870  Feature_118871  Feature_118872  \
0             0.0             0.0             0.0             0.0   
1             

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score
import pandas as pd

# Initializing MLP Classifier
clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, activation='relu', solver='adam', random_state=42)
print(clf)

# Training the model on training data
MLP = clf.fit(x_train_tfidf, y_train)

# Generating predictions on testing data
prediction = MLP.predict(x_test_tfidf)

# Measuring accuracy on testing data
print("Classification Report:")
print(classification_report(y_test, prediction))

print("Confusion Matrix:")
print(confusion_matrix(y_test, prediction))

# Calculating F1 Score
f1 = f1_score(y_test, prediction, average='weighted')
print('F1 Score:', round(f1, 2))

# Calculating Accuracy
accuracy = accuracy_score(y_test, prediction)
print('Accuracy:', round(accuracy, 2))

# Running 10-Fold Cross validation on MLP classifier
#Accuracy_Values = cross_val_score(clf, x_train_tfidf, y_train, cv=10, scoring='f1_weighted')
#print('\nAccuracy values for 10-fold Cross Validation:\n', Accuracy_Values)
#print('\nFinal Average Accuracy of the model:', round(Accuracy_Values.mean(), 2))




MLPClassifier(max_iter=400, random_state=42)
Classification Report:
              precision    recall  f1-score   support

       korku       0.94      0.91      0.92       921
   küçümseme       0.76      0.66      0.71       877
    mutluluk       0.82      0.82      0.82       908
        öfke       0.75      0.79      0.77       895
      üzüntü       0.74      0.81      0.77       920
   şaşkınlık       0.93      0.94      0.94       943

    accuracy                           0.82      5464
   macro avg       0.82      0.82      0.82      5464
weighted avg       0.83      0.82      0.82      5464

Confusion Matrix:
[[836   7   9  23  29  17]
 [ 19 583  74  95  89  17]
 [  7  50 748  39  54  10]
 [ 11  66  25 705  75  13]
 [  9  53  43  61 745   9]
 [  8   5   9  19  12 890]]
F1 Score: 0.82
Accuracy: 0.82


accurancy artırmak ıcın

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# Defining the parameter grid
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

# Setting up GridSearchCV
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='f1_weighted')

# Fitting the model
grid_search.fit(x_train_tfidf, y_train)

# Best model from grid search
best_clf = grid_search.best_estimator_

print(best_clf)

# Making predictions with the best model
prediction = best_clf.predict(x_test_tfidf)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

# Printing the Overall Accuracy of the model
F1_Score = metrics.f1_score(y_test, prediction, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score, 2))


MultinomialNB()
              precision    recall  f1-score   support

       korku       0.94      0.87      0.91       921
   küçümseme       0.77      0.64      0.70       877
    mutluluk       0.78      0.82      0.80       908
        öfke       0.79      0.74      0.76       895
      üzüntü       0.69      0.85      0.76       920
   şaşkınlık       0.90      0.91      0.91       943

    accuracy                           0.81      5464
   macro avg       0.81      0.80      0.80      5464
weighted avg       0.81      0.81      0.81      5464

[[805  10  12  25  43  26]
 [ 19 558  94  70 112  24]
 [  9  39 747  25  71  17]
 [  9  57  40 659 112  18]
 [  5  39  55  32 778  11]
 [ 11  22  14  18  17 861]]
Accuracy of the model on Testing Sample Data: 0.81
